<a href="https://colab.research.google.com/github/cptgingerbeard/good-fall/blob/main/excercise_pose_classifier_ido_video1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Modules

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install mediapipe
import cv2
import mediapipe as mp
print("MediaPipe imported successfully!")

import numpy as np
import pandas as pd
import tensorflow as tf
import keras

!pip install --upgrade numpy pandas tensorflow keras opencv-python mediapipe

In [ ]:
import cv2
import mediapipe as mp
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import plotly.graph_objects as go
import imageio
import pickle
#import seaborn as sns

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from google.colab.patches import cv2_imshow
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import VotingClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder

## Constants

In [ ]:
# Define all landmarks in uppercase
LANDMARKS = [
    "NOSE",
    "LEFT_EYE_INNER",
    "LEFT_EYE",
    "LEFT_EYE_OUTER",
    "RIGHT_EYE_INNER",
    "RIGHT_EYE",
    "RIGHT_EYE_OUTER",
    "LEFT_EAR",
    "RIGHT_EAR",
    "MOUTH_LEFT",
    "MOUTH_RIGHT",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_ELBOW",
    "RIGHT_ELBOW",
    "LEFT_WRIST",
    "RIGHT_WRIST",
    "LEFT_PINKY",
    "RIGHT_PINKY",
    "LEFT_INDEX",
    "RIGHT_INDEX",
    "LEFT_THUMB",
    "RIGHT_THUMB",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
    "LEFT_HEEL",
    "RIGHT_HEEL",
    "LEFT_FOOT_INDEX",
    "RIGHT_FOOT_INDEX"
]

BODY_ANGLES = ['left_elbow_angle',
 'right_elbow_angle',
 'left_armpit_angle',
 'right_armpit_angle',
 'left_hip_angle',
 'right_hip_angle',
 'left_knee_angle',
 'right_knee_angle',
 'left_side_collarbone_angle',
 'right_side_collarbone_angle']

BODY_HEIGHTS = [
    'shoulder_width_x', 'shoulder_width_y',
    'hip_width_x', 'hip_width_y',
    'left_elbow_wrist_distance_x', 'left_elbow_wrist_distance_y',
    'right_elbow_wrist_distance_x', 'right_elbow_wrist_distance_y',
    'left_elbow_shoulder_distance_x', 'left_elbow_shoulder_distance_y',
    'right_elbow_shoulder_distance_x', 'right_elbow_shoulder_distance_y',
    'left_shoulder_hip_alignment_x', 'left_shoulder_hip_alignment_y',
    'right_shoulder_hip_alignment_x', 'right_shoulder_hip_alignment_y',
    'left_knee_ankle_distance_x', 'left_knee_ankle_distance_y',
    'right_knee_ankle_distance_x', 'right_knee_ankle_distance_y',
    'left_knee_hip_distance_x', 'left_knee_hip_distance_y',
    'right_knee_hip_distance_x', 'right_knee_hip_distance_y',
    'left_hand_shoulder_distance_x', 'left_hand_shoulder_distance_y',
    'right_hand_shoulder_distance_x', 'right_hand_shoulder_distance_y',
    'left_hand_nose_distance_x', 'left_hand_nose_distance_y',
    'right_hand_nose_distance_x', 'right_hand_nose_distance_y',
    'left_hand_hip_distance_x', 'left_hand_hip_distance_y',
    'right_hand_hip_distance_x', 'right_hand_hip_distance_y',
    'foot_spread_x', 'foot_spread_y'
]


## Helper Functions

In [ ]:
# Helper function to get coordinates of a landmark
def get_coords(landmark, df_frame):
    coords = df_frame[df_frame['landmark'] == landmark][['x', 'y', 'z']].values
    return coords[0] if len(coords) > 0 else None

# Calculate vector between two points
def calc_vector(p1, p2):
    return np.array(p2) - np.array(p1)

# Calculate angle between two vectors using the dot product
def angle_between(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    cos_theta = dot_product / (norm_v1 * norm_v2)
    angle = np.arccos(np.clip(cos_theta, -1.0, 1.0))  # Clip to avoid numerical errors
    return np.degrees(angle)

def get_body_vecs(coordinates):
    # Upper arms
    left_upper_arm = calc_vector(coordinates['LEFT_SHOULDER'], coordinates['LEFT_ELBOW'])
    right_upper_arm = calc_vector(coordinates['RIGHT_SHOULDER'], coordinates['RIGHT_ELBOW'])

    # Forearms
    left_forearm = calc_vector(coordinates['LEFT_ELBOW'], coordinates['LEFT_WRIST'])
    right_forearm = calc_vector(coordinates['RIGHT_ELBOW'], coordinates['RIGHT_WRIST'])

    # Upper legs
    left_upper_leg = calc_vector(coordinates['LEFT_HIP'], coordinates['LEFT_KNEE'])
    right_upper_leg = calc_vector(coordinates['RIGHT_HIP'], coordinates['RIGHT_KNEE'])

    # Lower legs
    left_lower_leg = calc_vector(coordinates['LEFT_KNEE'], coordinates['LEFT_ANKLE'])
    right_lower_leg = calc_vector(coordinates['RIGHT_KNEE'], coordinates['RIGHT_ANKLE'])

    # Groin (hip-to-hip vector)
    groin = calc_vector(coordinates['LEFT_HIP'], coordinates['RIGHT_HIP'])

    # Collarbones (shoulder-to-shoulder vector)
    collarbones = calc_vector(coordinates['LEFT_SHOULDER'], coordinates['RIGHT_SHOULDER'])

    # Sides
    right_side = calc_vector(coordinates['RIGHT_SHOULDER'], coordinates['RIGHT_HIP'])
    left_side = calc_vector(coordinates['LEFT_SHOULDER'], coordinates['LEFT_HIP'])

    # Return all vectors in a dictionary
    return {
        "left_upper_arm": left_upper_arm,
        "right_upper_arm": right_upper_arm,
        "left_forearm": left_forearm,
        "right_forearm": right_forearm,
        "left_upper_leg": left_upper_leg,
        "right_upper_leg": right_upper_leg,
        "left_lower_leg": left_lower_leg,
        "right_lower_leg": right_lower_leg,
        "groin": groin,
        "collarbones": collarbones,
        "right_side": right_side,
        "left_side": left_side
    }

# Calculate Angles
def calculate_body_angles(vectors):
    # Elbow angles (between upper arm and forearm)
    left_elbow_angle = angle_between(vectors['left_upper_arm'], vectors['left_forearm'])
    right_elbow_angle = angle_between(vectors['right_upper_arm'], vectors['right_forearm'])

    # Armpit angles (between upper arm and collarbones)
    left_armpit_angle = angle_between(vectors['left_upper_arm'], vectors['collarbones'])
    right_armpit_angle = angle_between(vectors['right_upper_arm'], vectors['collarbones'])

    # Hip angles (between upper leg and side vectors, representing torso alignment)
    left_hip_angle = angle_between(vectors['left_upper_leg'], vectors['left_side'])
    right_hip_angle = angle_between(vectors['right_upper_leg'], vectors['right_side'])

    # Knee angles (between upper leg and lower leg)
    left_knee_angle = angle_between(vectors['left_upper_leg'], vectors['left_lower_leg'])
    right_knee_angle = angle_between(vectors['right_upper_leg'], vectors['right_lower_leg'])

    # Side to collarbone angles (lateral torso tilt)
    left_side_collarbone_angle = angle_between(vectors['left_side'], vectors['collarbones'])
    right_side_collarbone_angle = angle_between(vectors['right_side'], vectors['collarbones'])

    # Return all angles in a dictionary
    return {
        "left_elbow_angle": left_elbow_angle,
        "right_elbow_angle": right_elbow_angle,
        "left_armpit_angle": left_armpit_angle,
        "right_armpit_angle": right_armpit_angle,
        "left_hip_angle": left_hip_angle,
        "right_hip_angle": right_hip_angle,
        "left_knee_angle": left_knee_angle,
        "right_knee_angle": right_knee_angle,
        "left_side_collarbone_angle": left_side_collarbone_angle,
        "right_side_collarbone_angle": right_side_collarbone_angle
    }

# Helper function to calculate differences in x and y directions between two points
def calc_xy_difference(p1, p2):
    return p2[0] - p1[0], p2[1] - p1[1]

# Calculate Heights
def calculate_body_heights(coordinates):
    # Calculate relevant x and y differences
    heights = {
        # Shoulder width (left shoulder to right shoulder)
        "shoulder_width_x": calc_xy_difference(coordinates['LEFT_SHOULDER'], coordinates['RIGHT_SHOULDER'])[0],
        "shoulder_width_y": calc_xy_difference(coordinates['LEFT_SHOULDER'], coordinates['RIGHT_SHOULDER'])[1],

        # Hip width (left hip to right hip)
        "hip_width_x": calc_xy_difference(coordinates['LEFT_HIP'], coordinates['RIGHT_HIP'])[0],
        "hip_width_y": calc_xy_difference(coordinates['LEFT_HIP'], coordinates['RIGHT_HIP'])[1],

        # Elbow-Wrist distance for arms
        "left_elbow_wrist_distance_x": calc_xy_difference(coordinates['LEFT_ELBOW'], coordinates['LEFT_WRIST'])[0],
        "left_elbow_wrist_distance_y": calc_xy_difference(coordinates['LEFT_ELBOW'], coordinates['LEFT_WRIST'])[1],
        "right_elbow_wrist_distance_x": calc_xy_difference(coordinates['RIGHT_ELBOW'], coordinates['RIGHT_WRIST'])[0],
        "right_elbow_wrist_distance_y": calc_xy_difference(coordinates['RIGHT_ELBOW'], coordinates['RIGHT_WRIST'])[1],

        # Elbow-Shoulder distance for arms
        "left_elbow_shoulder_distance_x": calc_xy_difference(coordinates['LEFT_ELBOW'], coordinates['LEFT_SHOULDER'])[0],
        "left_elbow_shoulder_distance_y": calc_xy_difference(coordinates['LEFT_ELBOW'], coordinates['LEFT_SHOULDER'])[1],
        "right_elbow_shoulder_distance_x": calc_xy_difference(coordinates['RIGHT_ELBOW'], coordinates['RIGHT_SHOULDER'])[0],
        "right_elbow_shoulder_distance_y": calc_xy_difference(coordinates['RIGHT_ELBOW'], coordinates['RIGHT_SHOULDER'])[1],

        # Shoulder-Hip alignment for left and right sides
        "left_shoulder_hip_alignment_x": calc_xy_difference(coordinates['LEFT_SHOULDER'], coordinates['LEFT_HIP'])[0],
        "left_shoulder_hip_alignment_y": calc_xy_difference(coordinates['LEFT_SHOULDER'], coordinates['LEFT_HIP'])[1],
        "right_shoulder_hip_alignment_x": calc_xy_difference(coordinates['RIGHT_SHOULDER'], coordinates['RIGHT_HIP'])[0],
        "right_shoulder_hip_alignment_y": calc_xy_difference(coordinates['RIGHT_SHOULDER'], coordinates['RIGHT_HIP'])[1],

        # Knee-Ankle distance for legs
        "left_knee_ankle_distance_x": calc_xy_difference(coordinates['LEFT_KNEE'], coordinates['LEFT_ANKLE'])[0],
        "left_knee_ankle_distance_y": calc_xy_difference(coordinates['LEFT_KNEE'], coordinates['LEFT_ANKLE'])[1],
        "right_knee_ankle_distance_x": calc_xy_difference(coordinates['RIGHT_KNEE'], coordinates['RIGHT_ANKLE'])[0],
        "right_knee_ankle_distance_y": calc_xy_difference(coordinates['RIGHT_KNEE'], coordinates['RIGHT_ANKLE'])[1],

        # Knee-Hip distance for legs
        "left_knee_hip_distance_x": calc_xy_difference(coordinates['LEFT_KNEE'], coordinates['LEFT_HIP'])[0],
        "left_knee_hip_distance_y": calc_xy_difference(coordinates['LEFT_KNEE'], coordinates['LEFT_HIP'])[1],
        "right_knee_hip_distance_x": calc_xy_difference(coordinates['RIGHT_KNEE'], coordinates['RIGHT_HIP'])[0],
        "right_knee_hip_distance_y": calc_xy_difference(coordinates['RIGHT_KNEE'], coordinates['RIGHT_HIP'])[1],

        # Wrist-Shoulder distance for arms
        "left_hand_shoulder_distance_x": calc_xy_difference(coordinates['LEFT_WRIST'], coordinates['LEFT_SHOULDER'])[0],
        "left_hand_shoulder_distance_y": calc_xy_difference(coordinates['LEFT_WRIST'], coordinates['LEFT_SHOULDER'])[1],
        "right_hand_shoulder_distance_x": calc_xy_difference(coordinates['RIGHT_WRIST'], coordinates['RIGHT_SHOULDER'])[0],
        "right_hand_shoulder_distance_y": calc_xy_difference(coordinates['RIGHT_WRIST'], coordinates['RIGHT_SHOULDER'])[1],

        # Wrist-Nose distance
        "left_hand_nose_distance_x": calc_xy_difference(coordinates['LEFT_WRIST'], coordinates['NOSE'])[0],
        "left_hand_nose_distance_y": calc_xy_difference(coordinates['LEFT_WRIST'], coordinates['NOSE'])[1],
        "right_hand_nose_distance_x": calc_xy_difference(coordinates['RIGHT_WRIST'], coordinates['NOSE'])[0],
        "right_hand_nose_distance_y": calc_xy_difference(coordinates['RIGHT_WRIST'], coordinates['NOSE'])[1],

        # Wrist-Hip distance
        "left_hand_hip_distance_x": calc_xy_difference(coordinates['LEFT_WRIST'], coordinates['LEFT_HIP'])[0],
        "left_hand_hip_distance_y": calc_xy_difference(coordinates['LEFT_WRIST'], coordinates['LEFT_HIP'])[1],
        "right_hand_hip_distance_x": calc_xy_difference(coordinates['RIGHT_WRIST'], coordinates['RIGHT_HIP'])[0],
        "right_hand_hip_distance_y": calc_xy_difference(coordinates['RIGHT_WRIST'], coordinates['RIGHT_HIP'])[1],

        # Foot spread (left foot index to right foot index)
        "foot_spread_x": calc_xy_difference(coordinates['LEFT_FOOT_INDEX'], coordinates['RIGHT_FOOT_INDEX'])[0],
        "foot_spread_y": calc_xy_difference(coordinates['LEFT_FOOT_INDEX'], coordinates['RIGHT_FOOT_INDEX'])[1]
    }

    return heights


def calculate_angle(A, B, C):
    """
    Calculate the angle (in degrees) between vectors AB and BC.
    A, B, C are tuples representing (x, y) coordinates.
    """
    # Create vectors AB and BC
    AB = np.array([B[0] - A[0], B[1] - A[1]])
    BC = np.array([C[0] - B[0], C[1] - B[1]])

    # Calculate the dot product and magnitudes (norms) of the vectors
    dot_product = np.dot(AB, BC)
    magnitude_AB = np.linalg.norm(AB)
    magnitude_BC = np.linalg.norm(BC)

    # Calculate the cosine of the angle
    cos_theta = dot_product / (magnitude_AB * magnitude_BC)

    # Handle numerical precision issues (cosine should be between -1 and 1)
    cos_theta = np.clip(cos_theta, -1.0, 1.0)

    # Calculate the angle in radians and convert to degrees
    angle_rad = np.arccos(cos_theta)
    angle_deg = np.degrees(angle_rad)

    return angle_deg

def create_features_per_excercise(video_folder, excercise_type,mp_pose):
    excercise_path = os.path.join(video_folder, excercise_type)  # Construct the full path
    vid_count = 0
    csv_data = []
    if os.path.isdir(excercise_path):  # Ensure it's a directory
        for video_file in tqdm(os.listdir(excercise_path)):
            video_path = os.path.join(excercise_path, video_file)
            # print(f"Processing video: {video_path}")

            # Open video file
            cap = cv2.VideoCapture(video_path)

            if not cap.isOpened():
                print(f"Error: Could not open video {video_path}.")
                continue
            vid_count += 1
            frame_number = 0

            # Initialize MediaPipe Pose with default settings
            with mp_pose.Pose() as pose:
                while cap.isOpened():
                    ret, frame = cap.read()

                    if not ret:
                        # print("End of video or failed to read frame.")
                        break

                    # Convert the frame to RGB (MediaPipe works with RGB images)
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                    # Process the frame to detect pose landmarks
                    result = pose.process(frame_rgb)

                    # Draw pose landmarks if detected
                    if result.pose_landmarks:
                        for idx, landmark in enumerate(result.pose_landmarks.landmark):
                            # print(f"{mp_pose.PoseLandmark(idx).name}: (x: {landmark.x}, y: {landmark.y}, z: {landmark.z})")
                            csv_data.append([excercise_type,vid_count,frame_number, mp_pose.PoseLandmark(idx).name, landmark.x, landmark.y, landmark.z])
                    # Increment frame number
                    frame_number += 1
            # Release video capture and close all windows
            cap.release()
            cv2.destroyAllWindows()
    return csv_data

def feat_to_parquet(csv_data, excercise_name):
    df_feat = pd.DataFrame(csv_data)
    df_feat = df_feat.rename(columns={
        0: 'exc_name',
        1: 'vid_num',
        2: 'frame',
        3: 'landmark',
        4: 'x',
        5: 'y',
        6: 'z'
    })
    df_feat.to_parquet(f'{excercise_name}_features.parquet')

def pad_sequence(sequence_chunk, desired_sequence_length):
    num_rows_sequence = sequence_chunk.shape[0]
    num_columns_sequence = sequence_chunk.shape[1]

    # Determine the label value from the existing sequence
    label_value = sequence_chunk['exc_name_encoded'].iloc[0]  # Assumes the label is the same for the whole chunk

    # Create an empty sequence with the remaining rows, filled with 0
    empty_sequence = pd.DataFrame(
        np.full((desired_sequence_length - num_rows_sequence, num_columns_sequence), 0),
        columns=sequence_chunk.columns
    )

    # Set the label column in `empty_sequence` to the correct label
    empty_sequence['exc_name_encoded'] = label_value

    # Concatenate the original sequence with the padded rows
    padded_sequence = pd.concat([sequence_chunk, empty_sequence], axis=0).reset_index(drop=True)

    return padded_sequence

In [ ]:
# prompt: using calculate_speed check the speed of one landmark in 3d space between two frames 30 frames apart

def calculate_speed(frame1_landmarks, frame2_landmarks, frame_num_diff):
    """
    Calculates the speed of body landmarks between two frames.

    Args:
        frame1_landmarks (dict): Dictionary of landmark coordinates for the first frame.
        frame2_landmarks (dict): Dictionary of landmark coordinates for the second frame.
        frame_num_diff (int): Difference in frame numbers between the two frames.

    Returns:
        dict: A dictionary containing the speed of each landmark.
              Returns an empty dictionary if input is invalid or landmarks are missing.
    """

    if not isinstance(frame1_landmarks, dict) or not isinstance(frame2_landmarks, dict):
        print("Error: Input landmarks must be dictionaries.")
        return {}

    if not frame_num_diff or frame_num_diff <=0 :
      print("Error: frame number difference must be positive")
      return {}

    speeds = {}
    for landmark_name in frame1_landmarks:
        if landmark_name in frame2_landmarks:
            landmark1 = np.array(frame1_landmarks[landmark_name])
            landmark2 = np.array(frame2_landmarks[landmark_name])

            if landmark1.size == 3 and landmark2.size == 3:  # Check for 3D coordinates
                displacement = np.linalg.norm(landmark2 - landmark1)
                speed = displacement / frame_num_diff
                speeds[landmark_name] = speed
            else:
                print(f"Warning: Skipping landmark '{landmark_name}' - invalid coordinate dimensions.")
        else:
            print(f"Warning: Landmark '{landmark_name}' not found in both frames.")
    return speeds

# Example usage (replace with your actual data and frame numbers):
#frame_number_1 = 0  # Replace with the actual frame number
#frame_number_2 = 30 # Replace with frame 30 frames later

# Assuming you have already extracted landmark data into a dictionary for each frame
# Sample data below for the example
# Replace this with your actual dataframe and extracting the required information.

# Call calculate_speed with the landmarks and frame number difference:
#landmark_speeds = calculate_speed(frame1_landmarks, frame2_landmarks, 30)

In [ ]:
# prompt: take from df_trail_run the landmarks for a frame. use this frame1_landmarks = {
#     'LEFT_SHOULDER': (0.5, 0.6, 0.7),
#     'RIGHT_SHOULDER': (0.4, 0.7, 0.5),
#     'LEFT_ELBOW': (0.7, 0.3, 0.2)
#     # ... other landmarks
# }
# make a dictionary for each frame with this landmarks:### "NOSE",
# "LEFT_SHOULDER",
# "RIGHT_SHOULDER",
# "LEFT_ELBOW",
# "RIGHT_ELBOW",
# "LEFT_WRIST",
# "RIGHT_WRIST",
# "LEFT_HIP",
# "RIGHT_HIP",
# "LEFT_KNEE",
# "RIGHT_KNEE",
# "LEFT_ANKLE",
# "RIGHT_ANKLE",
# ###

def create_landmark_dict_for_frame(df_trail_run, frame_number):
  """
  Creates a dictionary of landmark coordinates for a given frame from the dataframe.

  Args:
      df_trail_run (pd.DataFrame): The dataframe containing landmark data.
      frame_number (int): The frame number to extract landmarks for.

  Returns:
      dict: A dictionary where keys are landmark names and values are (x, y, z) tuples.
  """

  df_frame = df_trail_run[df_trail_run['frame'] == frame_number]
  landmarks = {}
  for landmark_name in ['NOSE', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW', 'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE']:
    if not df_frame[df_frame['landmark'] == landmark_name].empty:
        x = df_frame[df_frame['landmark'] == landmark_name]['x'].values[0]
        y = df_frame[df_frame['landmark'] == landmark_name]['y'].values[0]
        z = df_frame[df_frame['landmark'] == landmark_name]['z'].values[0]
        landmarks[landmark_name] = (x, y, z)
  return landmarks



## Initial Mediapipe Trial

### Run landmark detection on a video

In [ ]:
# Initialize MediaPipe Pose and Drawing modules
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Video capture path (ensure it points to your video correctly)
video_path = r'/content/drive/MyDrive/videos_for_project/first_long_video_perperson.mp4'
excercise_type_temp = 'good fall'
vid_count_temp = 2
cap = cv2.VideoCapture(video_path)

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f'Total number of frames: {frame_count}')

# Get the frames per second (FPS)
fps = cap.get(cv2.CAP_PROP_FPS)
print(f'Frames per second (FPS): {fps}')

if not cap.isOpened():
    print(f"Error: Could not open video {video_path}.")
    exit()

frame_number = 0
csv_data = []
landmark_lst = []

# Initialize MediaPipe Pose with default settings
with mp_pose.Pose() as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video or failed to read frame.")
            break

        # Convert the frame to RGB (MediaPipe works with RGB images)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame to detect pose landmarks
        result = pose.process(frame_rgb)

        # Draw pose landmarks if detected
        if result.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS
            )

            frame_landmarks = result.pose_landmarks.landmark
            right_shoulder = (frame_landmarks[12].x,frame_landmarks[12].y)
            right_elbow = (frame_landmarks[14].x,frame_landmarks[14].y)
            right_wrist = (frame_landmarks[16].x,frame_landmarks[16].y)
            frame_angle = -(calculate_angle(right_wrist, right_elbow, right_shoulder) - 180)

            # Optional: Add landmark coordinates to CSV data
            for idx, landmark in enumerate(result.pose_landmarks.landmark):
                # print(f"{mp_pose.PoseLandmark(idx).name}: (x: {landmark.x}, y: {landmark.y}, z: {landmark.z})")
                csv_data.append([excercise_type_temp,vid_count_temp,frame_number, mp_pose.PoseLandmark(idx).name, landmark.x, landmark.y, landmark.z])
            landmark_lst.append(result.pose_landmarks)

            # Add text to the frame
        # text = f"Angle {round(frame_angle,2)}"
        # org = (50, 50)  # Coordinates for the text (x, y)
        # font = cv2.FONT_HERSHEY_SIMPLEX  # Font type
        # fontScale = 1  # Font size
        # color = (0, 255, 0)  # Green text in BGR
        # thickness = 2  # Thickness of the text
        # lineType = cv2.LINE_AA  # Anti-aliased text

        # # Write the text on the frame
        # cv2.putText(frame, text, org, font, fontScale, color, thickness, lineType)


        # cv2.putText(frame, frame_angle)
        # Display the frame in a window
        # cv2_imshow(frame)

        # Increment frame number
        frame_number += 1

        # Wait for 1ms and exit if 'q' is pressed
        #if cv2.waitKey(1) & 0xFF == ord('q'):
        #    break

    # Release video capture and close all windows
    cap.release()
    #cv2.destroyAllWindows()
    print("Video processing complete.")

### write to drive

In [ ]:
# Specify the path where you want to save the CSV file in your Google Drive
csv_file_path = '/content/drive/MyDrive/videos_for_project/csv_data/pose_landmarks_first_long_video.csv'

# Write the data to the CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    # Write the header row (if needed)
    csv_writer.writerow(['exc_name', 'vid_num', 'frame', 'landmark', 'x', 'y', 'z'])
    csv_writer.writerows(csv_data)

print(f"CSV data saved to: {csv_file_path}")


### read from drive

In [ ]:
csv_data = pd.read_csv('/content/drive/MyDrive/videos_for_project/csv_data/pose_landmarks_first_long_video.csv')

### Collect data for analysis

In [ ]:

df_trail_run = pd.DataFrame(csv_data)
# Rename the columns from 0-5 to meaningful names
df_trail_run = df_trail_run.rename(columns={
    0: 'exc_name',
    1: 'vid_num',
    2: 'frame',
    3: 'landmark',
    4: 'x',
    5: 'y',
    6: 'z'
})
df_trail_run

In [ ]:
# Look at what one frame looks like
df_trail_run[df_trail_run['frame'] == 200]

### Look at angles

right elbow angle over time

In [ ]:
frame_angles = []
for frame_num in np.unique(df_trail_run['frame']):
    right_elbow = (df_trail_run[(df_trail_run['frame'] == frame_num) & (df_trail_run['landmark'] == 'RIGHT_ELBOW')].x.values[0], df_trail_run[(df_trail_run['frame'] == frame_num) & (df_trail_run['landmark'] == 'RIGHT_ELBOW')].y.values[0])
    right_shoulder = (df_trail_run[(df_trail_run['frame'] == frame_num) & (df_trail_run['landmark'] == 'RIGHT_SHOULDER')].x.values[0],df_trail_run[(df_trail_run['frame'] == frame_num) & (df_trail_run['landmark'] == 'RIGHT_SHOULDER')].y.values[0])
    right_wrist = (df_trail_run[(df_trail_run['frame'] == frame_num) & (df_trail_run['landmark'] == 'RIGHT_WRIST')].x.values[0],df_trail_run[(df_trail_run['frame'] == frame_num) & (df_trail_run['landmark'] == 'RIGHT_WRIST')].y.values[0])
    frame_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
    frame_angles.append(frame_angle)

In [ ]:
plt.plot(range(len(frame_angles)),frame_angles)
plt.xlabel('Frame Number')
plt.ylabel('Angle')
plt.title('Angle Over Time')
plt.show()

### Working with the landmark list

In [ ]:
right_shoulder = (landmark_lst[0].landmark[12].x,landmark_lst[0].landmark[12].y)
right_elbow = (landmark_lst[0].landmark[14].x,landmark_lst[0].landmark[14].y)
right_wrist = (landmark_lst[0].landmark[16].x,landmark_lst[0].landmark[16].y)
frame_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
frame_angle

In [ ]:
# prompt: using  the landmark list and the helper function calculate_body_angles(). make a table of all the body angles in all frames and save it to my drive

# LANDMARKS is already defined globally

# Create an empty list to store angle data
all_body_angles = []

# Iterate over unique frames in the DataFrame
for frame_num in tqdm(np.unique(df_trail_run['frame']), desc="Processing frames"):
    df_frame = df_trail_run[df_trail_run['frame'] == frame_num]

    # Retrieve coordinates for all landmarks in a dictionary
    coordinates = {landmark: get_coords(landmark, df_frame) for landmark in LANDMARKS}

    # Calculate body vectors using the coordinates
    vectors = get_body_vecs(coordinates)

    # Calculate frame angles using the vectors
    frame_angles = calculate_body_angles(vectors)

    frame_angles['frame'] = frame_num
    all_body_angles.append(frame_angles)

# Convert to dataframe and save
angle_df = pd.DataFrame(all_body_angles)
angle_df.to_csv('/content/drive/MyDrive/body_angles_first_long_video.csv', index=False)

### 3d plot

###read csv data from file and DBSCAN to detect fall in frames

In [ ]:

# קריאת הנתונים
file_path = "/content/drive/MyDrive/videos_for_project/csv_data/body_angles_first_long_video.csv"
df = pd.read_csv(file_path)

# הסרת עמודת ה-frame לצורך עיבוד
frame_col = df['frame']  # שמירת מספרי הפריימים
features = df.drop(columns=['frame'])

# נורמליזציה של הנתונים
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)


dbscan = DBSCAN(eps=0.9, min_samples=5)
df['cluster'] = dbscan.fit_predict(features_scaled)

# סינון הנתונים כך שנראה רק את הקבוצות -1 (חריגים) ו-0
df_filtered = df[df['cluster'].isin([-1, 0])]

# שחזור עמודת ה-frame
df['frame'] = frame_col

# איתור פריימים שבהם התחולל מעבר בין עמידה לרצפה (או להפך)
df['transition'] = df['cluster'].diff().abs().fillna(0)
transition_frames = df[df['transition'] == 1]['frame'].tolist()

# הצגת פריימים שבהם התחולל מעבר בין מצבים
print("Transition Frames (Fall Start Detected):", transition_frames)

# ויזואליזציה - הצגת הזוויות בפריימים מסודרים לפי הזמן
plt.figure(figsize=(12, 6))
sns.scatterplot(
    x=df_filtered["frame"], y=df_filtered["cluster"],
    hue=df_filtered["cluster"], palette=["red", "blue"], alpha=0.7
)
plt.xlabel("Frame")
plt.ylabel("Cluster (-1=Outlier/Fall, 0=Standing)")
plt.title("DBSCAN Clustering: Detecting Falls (-1) vs. Standing (0)")
plt.legend(title="Cluster")
plt.show()

In [ ]:
df_filtered

#### Initial 3d plot

In [ ]:
# Initialize MediaPipe Pose module
mp_pose = mp.solutions.pose
POSE_CONNECTIONS = mp_pose.POSE_CONNECTIONS
frame_number = 5

# Filter the data for the desired frame
df_frame = df_trail_run[df_trail_run['frame'] == frame_number]

prediction = df_filtered[df_filtered['frame'] == frame_number]['cluster'].values[0]

# Create a dictionary mapping landmark names to their (x, y, z) coordinates
landmarks = {row['landmark']: (row['x'], row['y'], row['z']) for _, row in df_frame.iterrows()}

# Initialize the 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot landmarks as scatter points
for name, (x, y, z) in landmarks.items():
    ax.scatter(-z, x, y, label=name, s=50)  # Adjusting axes to align with desired orientation

# Helper function to get coordinates by landmark name
def get_coord(name):
    """Returns the (x, y, z) coordinates of a given landmark."""
    return landmarks.get(name, (None, None, None))

# Plot lines connecting the landmarks using POSE_CONNECTIONS
for connection in POSE_CONNECTIONS:
    start, end = connection
    start_name = mp_pose.PoseLandmark(start).name
    end_name = mp_pose.PoseLandmark(end).name

    # Get the coordinates of the two landmarks
    start_coord = get_coord(start_name)
    end_coord = get_coord(end_name)

    # Check if both landmarks are available
    if None not in start_coord and None not in end_coord:
        ax.plot(
            [-start_coord[2], -end_coord[2]],  # x-coordinates
            [start_coord[0], end_coord[0]],    # y-coordinates
            [start_coord[1], end_coord[1]],    # z-coordinates
            color='blue'
        )

# Set axis labels
ax.set_zlabel('y')
ax.set_xlabel('z')
ax.set_ylabel('x')

# Flip the Z-axis to ensure head is at the top and legs at the bottom
ax.invert_zaxis()

# Rotate, tilt, and twist the graph
ax.view_init(elev=10, azim=20)  # Adjust 'elev' and 'azim' as needed

# Set plot title
plt.title(f'3D Pose for Frame {frame_number} prediction: {prediction}')

# Display the plot
plt.show()

#### Interactive 3d plot

In [ ]:
# Initialize MediaPipe Pose module
mp_pose = mp.solutions.pose
POSE_CONNECTIONS = mp_pose.POSE_CONNECTIONS
frame_number = 161

# Filter the data for the desired frame
df_frame = df_trail_run[df_trail_run['frame'] == frame_number]

# Create a dictionary mapping landmark names to their (x, y, z) coordinates
landmarks = {row['landmark']: (row['x'], row['y'], row['z']) for _, row in df_frame.iterrows()}

# Prepare lists to store points and connections
x_vals, y_vals, z_vals, names = [], [], [], []

# Collect the landmarks for plotting
for name, (x, y, z) in landmarks.items():
    x_vals.append(x)
    y_vals.append(y)
    z_vals.append(z)
    names.append(name)

# Create a 3D scatter plot for landmarks
scatter = go.Scatter3d(
    x=x_vals, y=z_vals, z=y_vals,  # Swap axes to align with your desired orientation
    mode='markers+text',
    marker=dict(size=5, color='blue'),
    text=names,  # Show names as text labels
    textposition="top center"
)

# Create line segments for each connection
connections = []
for connection in POSE_CONNECTIONS:
    start, end = connection
    start_name = mp_pose.PoseLandmark(start).name
    end_name = mp_pose.PoseLandmark(end).name

    start_coord = landmarks.get(start_name, None)
    end_coord = landmarks.get(end_name, None)

    # Ensure both landmarks exist
    if start_coord and end_coord:
        connections.append(
            go.Scatter3d(
                x=[start_coord[0], end_coord[0]],
                y=[start_coord[2], end_coord[2]],  # Swap Y and Z axes
                z=[start_coord[1], end_coord[1]],
                mode='lines',
                line=dict(color='blue', width=2)
            )
        )

# Combine scatter plot and connections into a figure
fig = go.Figure(data=[scatter] + connections)

# Set axis labels and layout
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Z',
        zaxis_title='Y',
        zaxis=dict(autorange='reversed')  # Ensure head is at the top and legs at the bottom
    ),
    title=f"3D Pose for Frame {frame_number}",
    showlegend=False
)

# Display the interactive plot
fig.show()


###create a dataframe with the landmarks as columns and the speed as rows. add the speed for every frame as a row

In [ ]:
# prompt: create a dataframe with the landmarks as columns and the speed as rows. add the speed for every frame as a row

# Assuming you have the necessary variables and functions defined (like LANDMARKS, df_trail_run)

# Create an empty list to store speed data for each frame
all_landmark_speeds = []

# Iterate through frames, calculating speed between consecutive frames
for frame_num in tqdm(range(len(np.unique(df_trail_run['frame'])) - 1), desc="Calculating speeds"):
  frame1_landmarks = create_landmark_dict_for_frame(df_trail_run, frame_num)
  frame2_landmarks = create_landmark_dict_for_frame(df_trail_run, frame_num + 30)

  landmark_speeds = calculate_speed(frame1_landmarks, frame2_landmarks, 30)  # Frame difference is 30

  landmark_speeds['frame'] = frame_num
  frame_num = frame_num + 30
  all_landmark_speeds.append(landmark_speeds)


# Convert to DataFrame
speed_df = pd.DataFrame(all_landmark_speeds)

# Reorder columns so 'frame' is first
cols = speed_df.columns.tolist()
cols.remove('frame')
speed_df = speed_df[['frame'] + cols]

# Display the DataFrame (optional)
speed_df


In [ ]:
# Specify the path where you want to save the CSV file in your Google Drive
csv_file_path = '/content/drive/MyDrive/videos_for_project/csv_data/speeds_landmarks_first_long_video.csv'

# Write the data to the CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    # Write the header row (if needed)
    csv_writer.writerow(speed_df.columns)
    # Write the data rows
    csv_writer.writerows(speed_df.values)

print(f"CSV data saved to: {csv_file_path}")


In [ ]:
# prompt: Using dataframe speed_df: view one landmark as a graph when x is time y is speed

# Import necessary libraries
import matplotlib.pyplot as plt

# Select a specific landmark (e.g., 'NOSE') and plot its speed over time
landmark_name = 'LEFT_WRIST'

# Assuming 'frame' represents time and the landmark column represents speed
plt.plot(speed_df['frame'], speed_df[landmark_name])

# Customize the plot
plt.xlabel('Time (frame)')
plt.ylabel('Speed')
plt.title(f'Speed of {landmark_name} Over Time')

# Show the plot
plt.show()


In [ ]:
# prompt: from the dataframe speed_df calculate the acceleration of each landmark

# Calculate acceleration for each landmark
acceleration_df = speed_df.copy()
for landmark in speed_df.columns[1:]:
  acceleration_df[landmark + '_acceleration'] = acceleration_df[landmark].diff()

# Display the DataFrame with acceleration (optional)
acceleration_df


In [ ]:
# prompt: make  plot for one landmark speed and acceleration

import matplotlib.pyplot as plt
# Select a specific landmark (e.g., 'NOSE') and plot its speed and acceleration over time
landmark_name = 'LEFT_WRIST'

# Assuming 'frame' represents time and the landmark column represents speed
plt.figure(figsize=(12, 6))
plt.plot(speed_df['frame'], speed_df[landmark_name], label='Speed')
plt.plot(acceleration_df['frame'], acceleration_df[landmark_name + '_acceleration'], label='Acceleration')

# Customize the plot
plt.xlabel('Time (frame)')
plt.ylabel('Speed / Acceleration')
plt.title(f'Speed and Acceleration of {landmark_name} Over Time')
plt.legend()

# Show the plot
plt.show()


In [ ]:

# קריאת הנתונים
#file_path = "/content/drive/MyDrive/videos_for_project/csv_data/body_angles_first_long_video.csv"
#df = pd.read_csv(file_path)
df = speed_df
# הסרת עמודת ה-frame לצורך עיבוד
frame_col = df['frame']  # שמירת מספרי הפריימים
features = df.drop(columns=['frame'])
# remove NaN
features = features.fillna(0)

# נורמליזציה של הנתונים
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)


dbscan = DBSCAN(eps=0.9, min_samples=5)
df['cluster'] = dbscan.fit_predict(features_scaled)

# סינון הנתונים כך שנראה רק את הקבוצות -1 (חריגים) ו-0
df_filtered = df[df['cluster'].isin([-1, 0])]

# שחזור עמודת ה-frame
df['frame'] = frame_col

# איתור פריימים שבהם התחולל מעבר בין עמידה לרצפה (או להפך)
df['transition'] = df['cluster'].diff().abs().fillna(0)
transition_frames = df[df['transition'] == 1]['frame'].tolist()

# הצגת פריימים שבהם התחולל מעבר בין מצבים
print("Transition Frames (Fall Start Detected):", transition_frames)

# ויזואליזציה - הצגת הזוויות בפריימים מסודרים לפי הזמן
plt.figure(figsize=(12, 6))
sns.scatterplot(
    x=df_filtered["frame"], y=df_filtered["cluster"],
    hue=df_filtered["cluster"], palette=["red", "blue"], alpha=0.7
)
plt.xlabel("Frame")
plt.ylabel("Cluster (-1=Outlier/Fall, 0=Standing)")
plt.title("DBSCAN Clustering: Detecting Falls (-1) vs. Standing (0)")
plt.legend(title="Cluster")
plt.show()

#### fall detection from body angles and speeds

In [ ]:
body_angles_path ="/content/drive/MyDrive/videos_for_project/csv_data/first long video/body_angles_first_long_video.csv"
body_speeds_path ="/content/drive/MyDrive/videos_for_project/csv_data/first long video/speeds_landmarks_first_long_video.csv"
landmarks_path ="/content/drive/MyDrive/videos_for_project/csv_data/first long video/pose_landmarks_first_long_video.csv"

In [ ]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster

def detect_falls(angles_path, speeds_path, landmarks_path, intensity_threshold_first_half=180,
                 intensity_threshold_second_half=200, min_falls=8, max_falls=12):
    # Load data
    angles_df = pd.read_csv(angles_path)
    speeds_df = pd.read_csv(speeds_path)
    landmarks_df = pd.read_csv(landmarks_path)

    # Merge datasets
    merged_df = angles_df.merge(speeds_df, on="frame", how="inner")
    landmarks_filtered = landmarks_df[['frame']].drop_duplicates()
    final_df = merged_df.merge(landmarks_filtered, on="frame", how="inner")

    # Handle missing values
    final_df.fillna(final_df.mean(), inplace=True)

    # Compute movement intensity
    movement_diff = final_df.drop(columns=['frame']).diff().abs().sum(axis=1)
    final_df['movement_intensity'] = movement_diff

    # Predict fall frames based on intensity
    predicted_fall_frames = final_df[final_df['movement_intensity'] > min(intensity_threshold_first_half,
                                                                          intensity_threshold_second_half)]['frame'].values

    # Split frames by first and second half
    mid_frame = final_df['frame'].median()
    first_half = predicted_fall_frames[predicted_fall_frames < mid_frame]
    second_half = predicted_fall_frames[predicted_fall_frames >= mid_frame]

    # Filter based on respective intensity thresholds
    first_half_filtered = [f for f in first_half if final_df.loc[f, 'movement_intensity'] > intensity_threshold_first_half]
    second_half_filtered = [f for f in second_half if final_df.loc[f, 'movement_intensity'] > intensity_threshold_second_half]
    all_filtered = np.array(first_half_filtered + second_half_filtered).reshape(-1, 1)

    # Cluster fall frames
    def find_optimal_threshold(frames, min_falls, max_falls):
        for t in np.linspace(150, 20, 20):
            Z = linkage(frames, method='ward')
            clusters = fcluster(Z, t=t, criterion='distance')
            if min_falls <= len(np.unique(clusters)) <= max_falls:
                return clusters
        return fcluster(Z, t=50, criterion='distance')

    if len(all_filtered) > 1:
        clusters = find_optimal_threshold(all_filtered, min_falls, max_falls)
        clustered_df = pd.DataFrame({'frame': all_filtered.flatten(), 'cluster': clusters})
        fall_starts = clustered_df.groupby('cluster')['frame'].min().sort_values().reset_index(drop=True)
    else:
        fall_starts = pd.Series([], dtype=int)

    return fall_starts

# Example usage:
# fall_frames = detect_falls("body_angles.csv", "body_speeds.csv", "landmarks.csv")
# fall_frames.to_csv("refined_fall_detection_results.csv", index=False)

fall_frames = detect_falls(body_angles_path, body_speeds_path, landmarks_path,intensity_threshold_second_half=100)
#fall_frames.to_csv("refined_fall_detection_results.csv", index=False)
fall_frames

In [ ]:
df_trail_run = pd.read_csv(landmarks_path)

In [ ]:
# Initialize MediaPipe Pose module
mp_pose = mp.solutions.pose
POSE_CONNECTIONS = mp_pose.POSE_CONNECTIONS
frame_number = 98

# Filter the data for the desired frame
df_frame = df_trail_run[df_trail_run['frame'] == frame_number]

# Create a dictionary mapping landmark names to their (x, y, z) coordinates
landmarks = {row['landmark']: (row['x'], row['y'], row['z']) for _, row in df_frame.iterrows()}

# Initialize the 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot landmarks as scatter points
for name, (x, y, z) in landmarks.items():
    ax.scatter(-z, x, y, label=name, s=50)  # Adjusting axes to align with desired orientation

# Helper function to get coordinates by landmark name
def get_coord(name):
    """Returns the (x, y, z) coordinates of a given landmark."""
    return landmarks.get(name, (None, None, None))

# Plot lines connecting the landmarks using POSE_CONNECTIONS
for connection in POSE_CONNECTIONS:
    start, end = connection
    start_name = mp_pose.PoseLandmark(start).name
    end_name = mp_pose.PoseLandmark(end).name

    # Get the coordinates of the two landmarks
    start_coord = get_coord(start_name)
    end_coord = get_coord(end_name)

    # Check if both landmarks are available
    if None not in start_coord and None not in end_coord:
        ax.plot(
            [-start_coord[2], -end_coord[2]],  # x-coordinates
            [start_coord[0], end_coord[0]],    # y-coordinates
            [start_coord[1], end_coord[1]],    # z-coordinates
            color='blue'
        )

# Set axis labels
ax.set_zlabel('y')
ax.set_xlabel('z')
ax.set_ylabel('x')

# Flip the Z-axis to ensure head is at the top and legs at the bottom
ax.invert_zaxis()

# Rotate, tilt, and twist the graph
ax.view_init(elev=10, azim=20)  # Adjust 'elev' and 'azim' as needed

# Set plot title
plt.title(f'3D Pose for Frame {frame_number}')

# Display the plot
plt.show()

#### Turn 3d plot into movie with multiple frames

In [ ]:
# Initialize MediaPipe Pose module
mp_pose = mp.solutions.pose
POSE_CONNECTIONS = mp_pose.POSE_CONNECTIONS

# Create a directory to store individual frame images
os.makedirs('frames', exist_ok=True)

# Determine the overall min and max values across all frames to set consistent axis limits
x_min, x_max = df_trail_run['x'].min(), df_trail_run['x'].max()
y_min, y_max = df_trail_run['y'].min(), df_trail_run['y'].max()
z_min, z_max = df_trail_run['z'].min(), df_trail_run['z'].max()

def get_coord(name, landmarks):
    """Returns the (x, y, z) coordinates of a given landmark."""
    return landmarks.get(name, (None, None, None))

# Generate and save plots for each frame
for frame_number in range(df_trail_run['frame'].max() + 1):
    # Filter the data for the desired frame
    df_trail_run_frame = df_trail_run[df_trail_run['frame'] == frame_number]

    if df_trail_run_frame.empty:
      # Handle the case where there are no predictions for this frame
      # You might want to skip this frame, assign a default value, or raise an exception
      print(f"Warning: No predictions found for frame {frame_number}. Skipping this frame.")
      continue  # Skip to the next frame
    prediction = df[df['frame'] == frame_number]['cluster'].values[0]



    # Create a dictionary mapping landmark names to their (x, y, z) coordinates
    landmarks = {row['landmark']: (row['x'], row['y'], row['z']) for _, row in df_trail_run_frame.iterrows()}

    # Initialize the 3D plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Plot landmarks as scatter points
    for name, (x, y, z) in landmarks.items():
        ax.scatter(-z, x, y, label=name, s=50)  # Adjusting axes to align with desired orientation

    # Plot lines connecting the landmarks using POSE_CONNECTIONS
    for connection in POSE_CONNECTIONS:
        start, end = connection
        start_name = mp_pose.PoseLandmark(start).name
        end_name = mp_pose.PoseLandmark(end).name

        start_coord = get_coord(start_name, landmarks)
        end_coord = get_coord(end_name, landmarks)

        # Check if both landmarks are available
        if None not in start_coord and None not in end_coord:
            ax.plot(
                [-start_coord[2], -end_coord[2]],  # x-coordinates
                [start_coord[0], end_coord[0]],    # y-coordinates
                [start_coord[1], end_coord[1]],    # z-coordinates
                color='blue'
            )

    # Set consistent axis limits
    ax.set_xlim([-z_max, -z_min])
    ax.set_ylim([x_min, x_max])
    ax.set_zlim([y_min, y_max])

    # Set axis labels
    ax.set_xlabel('Z')
    ax.set_ylabel('X')
    ax.set_zlabel('Y')

    # Flip the Z-axis to ensure head is at the top and legs at the bottom
    ax.invert_zaxis()

    # Maintain aspect ratio
    ax.set_box_aspect([1, 1, 1])  # Same scaling for all axes

    # Rotate, tilt, and twist the graph
    ax.view_init(elev=10, azim=20)  # Adjust 'elev' and 'azim' as needed

    # Set plot title
    plt.title(f'3D Pose for Frame {frame_number} prediction: {prediction}')


    # Save the plot as an image
    filename = f'frames/frame_{frame_number:03d}.png'
    plt.savefig(filename)
    plt.close(fig)  # Close the figure to free memory

# Create a video from the saved images
with imageio.get_writer('pose_video_with_prediction.mp4', fps=10) as writer:
    for frame_number in range(df_trail_run['frame'].max() + 1):
        filename = f'frames/frame_{frame_number:03d}.png'
        if not os.path.exists(filename):
            continue  # Skip if the file doesn't exist
        image = imageio.imread(filename)
        writer.append_data(image)

print("Video saved as 'pose_video_with_prediction.mp4'")

In [ ]:
# Create a video from the saved images
with imageio.get_writer('pose_video_3.mp4', fps=10) as writer:
    for frame_number in range(df_trail_run['frame'].max() + 1):
        filename = f'frames/frame_{frame_number:03d}.png'
        if not os.path.exists(filename):
            continue  # Skip if the file doesn't exist
        image = imageio.imread(filename)
        writer.append_data(image)

print("Video saved as 'pose_video_3.mp4'")